# setup

Execute:

```bash
docker pull docker pull segaleran/opencv-jupyter
docker run --rm -it -p 8888:8888/tcp \
 -v /path/to/workspace:/home/jovyan/work \
  segaleran/opencv-jupyter:latest
```

And then access the notebook at: http://127.0.0.1:8888/tree

# version check

In [ ]:
import sys

assert sys.version_info[:2] == (3, 5)

# install dependecies
## pytorch 1.0.0

In [ ]:
!wget -nc -P /tmp/ https://download.pytorch.org/whl/cpu/torch-1.0.0-cp35-cp35m-linux_x86_64.whl
!pip3 install /tmp/torch-1.0.0-cp35-cp35m-linux_x86_64.whl

## other APIs

In [ ]:
!pip3 install pillow==5.4.1 h5py==2.8.0 tqdm==4.30.0 #numpy==1.15.4 opencv-python==4.5.1.48

# code

## parameters

Feel free to choose any scale from: 2, 3, 4  
but remember to change the number behind 'x', before '.pth' in the download link, aswell as the scale value.

In [ ]:
#this may take a few attempts, just wait ;)
!wget -nc -P /tmp/ https://www.dropbox.com/s/rxluu1y8ptjm4rn/srcnn_x2.pth

In [ ]:
scale           = 2;
weights_file    = "/tmp/srcnn_x%d.pth" % scale;
image_file      = "../example_image.png"

In [ ]:
import torch
import cv2

device = "cpu";

## model

In [ ]:
from torch import nn

class SRCNN(nn.Module):
    def __init__(self, num_channels=1):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=9, padding=9 // 2)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=5 // 2)
        self.conv3 = nn.Conv2d(32, num_channels, kernel_size=5, padding=5 // 2)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x

In [ ]:
model = SRCNN().to(device);

state_dict = model.state_dict()
for n, p in torch.load(weights_file, map_location=lambda storage, _: storage).items():
    if n in state_dict.keys():
        state_dict[n].copy_(p)
    else:
        raise KeyError(n)

model.eval()

## images

In [ ]:
original = cv2.imread(image_file);
if (original is None):
    raise RuntimeError("Can't open image.")

downscaled = cv2.resize(original, dsize=None, fx=1/scale, fy=1/scale);

### upscaling

In [ ]:
upscaled = cv2.resize(downscaled, dsize=None, fx=scale, fy=scale);
upscaled_ycrcb = cv2.cvtColor(upscaled, cv2.COLOR_BGR2YCrCb);

### super sampling

In [ ]:
import numpy as np

upscaled_ycrcb = np.asarray(upscaled_ycrcb[:,:]).astype(np.float32);

y = upscaled_ycrcb[..., 0]
y /= 255.
y = torch.from_numpy(y).to(device)
y = y.unsqueeze(0).unsqueeze(0)

In [ ]:
with torch.no_grad():
    preds = model(y).clamp(0.0, 1.0)
preds = preds.mul(255.0).cpu().numpy().squeeze(0).squeeze(0)

In [ ]:
output = np.array([preds, upscaled_ycrcb[..., 1], upscaled_ycrcb[..., 2]]).transpose([1, 2, 0])
output = np.clip(output, 0.0, 255.0).astype(np.uint8)

output = cv2.cvtColor(output, cv2.COLOR_YCrCb2BGR);

## display results

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#https://stackoverflow.com/a/42314798
def display_image_in_actual_size(im_data):
    dpi = 80
    height, width, depth = im_data.shape

    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(im_data, cmap='gray')

    plt.show()

### original

In [ ]:
display_image_in_actual_size(original[:,:,::-1])

### SRCNN upscaled

In [ ]:
display_image_in_actual_size(output[:,:,::-1])

### opencv upscaled

In [ ]:
display_image_in_actual_size(upscaled[:,:,::-1])